In [102]:
import pandas as pd
from pathlib import Path

file_name = "best_data.csv"              
df = pd.read_csv(Path.cwd() / file_name)

df_clean = df.dropna()

print(f"Rows after dropping nulls: {len(df_clean):,}")


Rows after dropping nulls: 1,066


In [103]:
df_clean = df_clean.rename(columns={'property_type': 'Tipo de vivienda'})

In [104]:
import numpy as np

np.random.seed(42)

# 1. Compute masks for the three deterministic cases
mask_needs_reno   = df_clean['needs renovating']                              # True  → “a reformar”
mask_new_build    = ~df_clean['second hand'] & ~df_clean['needs renovating']        # False/False → “a estrenar”
mask_second_good  = df_clean['second hand'] & ~df_clean['needs renovating']         # True/False  → random choice

# 2. Allocate the values
df_clean.loc[mask_needs_reno, 'Estado'] = 'a reformar'
df_clean.loc[mask_new_build,  'Estado'] = 'a estrenar'

# For the rows that are second-hand but in good condition, choose randomly
choices = np.random.choice(['en buen estado', 'reformado'],
                           size=mask_second_good.sum())
df_clean.loc[mask_second_good, 'Estado'] = choices

# 3. Drop the original boolean columns
df_clean = df_clean.drop(columns=['second hand', 'needs renovating'])

# 4. (Optional) inspect the result
print(df_clean['Estado'].value_counts())


Estado
reformado         478
en buen estado    465
a reformar        123
Name: count, dtype: int64


In [105]:
# Count rows whose 'adress' cell contains the substring "calle"
n_calle = df_clean['adress'].str.contains('calle', case=False, na=False).sum()

print(f"Rows with 'calle' in the adress column: {n_calle}")


Rows with 'calle' in the adress column: 607


In [106]:
df_clean.rename(columns={'n_bedrooms': 'Habitaciones',
                         'bathrooms':  'Baños'},
                inplace=True)


In [107]:
import numpy as np
import re
from datetime import datetime

# ── tweak this only ──────────────────────────────────────────────────────────
RENT_THRESHOLD = 20_000        # €: anything below this is treated as rent
# ---------------------------------------------------------------------------

current_year = datetime.now().year

pat_new  = re.compile(r'\b(obra\s+nueva|new\s+(build|development)|obra_nueva)\b', re.I)
pat_rent = re.compile(r'\b(alquiler|alquilar|se\s+alquila|en\s+alquiler|for\s+rent|to\s+rent)\b', re.I)

def classify_interes(row):
    text  = str(row['description']).lower()
    price = row['price']

    # ---- obra nueva ---------------------------------------------------------
    if pat_new.search(text):
        return 'obra nueva'
    if not row.get('second hand', True) and not row.get('needs renovating', False):
        return 'obra nueva'
    if row.get('year_built') is not None and row['year_built'] >= current_year - 2:
        return 'obra nueva'

    # ---- alquiler -----------------------------------------------------------
    if pat_rent.search(text) or price < RENT_THRESHOLD:
        return 'alquilar'

    # ---- comprar (fallback) -------------------------------------------------
    return 'comprar'

# apply & split
df_clean['Interes'] = df_clean.apply(classify_interes, axis=1)

df_clean['Precio de compra (€)'] = np.where(
    df_clean['Interes'].isin(['comprar', 'obra nueva']),
    df_clean['price'],
    np.nan
)
df_clean['Precio de alquiler'] = np.where(
    df_clean['Interes'] == 'alquilar',
    df_clean['price'],
    np.nan
)

# (optional) keep or drop 'price' as you prefer
# df_clean.drop(columns=['price'], inplace=True)

print(df_clean['Interes'].value_counts(dropna=False))



Interes
comprar       1023
obra nueva      27
alquilar        16
Name: count, dtype: int64


In [108]:
import numpy as np

# Parameters
n_to_convert = 350
price_to_rent_years = 20

# Filter only "comprar" rows with non-null purchase price
comprar_mask = (df_clean['Interes'] == 'comprar') & df_clean['Precio de compra (€)'].notnull()

# Randomly sample 350 indices from that set
indices_to_convert = df_clean[comprar_mask].sample(n=n_to_convert, random_state=42).index

# Update 'Interes' to 'alquilar'
df_clean.loc[indices_to_convert, 'Interes'] = 'alquilar'

# Calculate rental price and set it
df_clean.loc[indices_to_convert, 'Precio de alquiler'] = (
    df_clean.loc[indices_to_convert, 'Precio de compra (€)'] / (price_to_rent_years * 12)
)

# Nullify 'Precio de compra (€)'
df_clean.loc[indices_to_convert, 'Precio de compra (€)'] = np.nan


In [109]:
df_clean.rename(columns={'sq_mt_built': 'Superficie total (m²)'}, inplace=True)


In [110]:
df_clean.rename(columns={'heating': 'Calefacción'}, inplace=True)


In [111]:
df_clean.rename(columns={'air conditioning': 'Aire acondicionado'}, inplace=True)


In [112]:
df_clean.rename(columns={'parking': 'garage'}, inplace=True)


In [113]:
df_clean.rename(columns={'balcony': 'Balcon'}, inplace=True)


In [114]:
df_clean.rename(columns={'terrace': 'Terraza'}, inplace=True)


In [115]:
df_clean.rename(columns={'swimming pool': 'Piscina'}, inplace=True)


In [116]:
df_clean.rename(columns={'garden': 'Jardin'}, inplace=True)


In [117]:
df_clean.rename(columns={'lift_x': 'Ascensor'}, inplace=True)


In [118]:
import numpy as np

# 1️⃣  Keep the numeric storey as-is
df_clean['Numero de Planta'] = df_clean['floor_y']

# 2️⃣  Map floor_y → Tipo de planta
df_clean['Tipo de planta'] = np.select(
    condlist=[
        df_clean['floor_y'] <= 0,                  # Baja: ≤ 0
        df_clean['floor_y'].between(1, 5),        # Intermedia: 1-5
        df_clean['floor_y'] >= 6                  # Alta: ≥ 6
    ],
    choicelist=['Baja', 'Intermedia', 'Alta'],
    default=None          # avoids the dtype clash with strings
).astype('object')        # make the column a clean object/str dtype

# 3️⃣  Remove the old column
df_clean.drop(columns=['floor_y'], inplace=True)

# 4️⃣  (Optional) quick check
print(df_clean['Tipo de planta'].value_counts(dropna=False))


Tipo de planta
Intermedia    787
Baja          143
Alta          136
Name: count, dtype: int64


In [119]:
df_clean.rename(columns={'exterior_x': 'Vistas al exterior'}, inplace=True)


In [120]:
df_clean.rename(columns={'sq_m_floor_area': 'Superficie util (m2)'}, inplace=True)


In [121]:
import numpy as np
import pandas as pd

# ─────────────────────────────────────────────────────────────────────────────
# 0.  IDAE reference bands (housing, national averages)
#     “≤ upper-bound” belongs to that band
# ─────────────────────────────────────────────────────────────────────────────
cons_thresholds  = np.array([44.6, 72.3, 112.1, 172.3, 303.7, 382.6])   # kWh
emis_thresholds  = np.array([10,   16.3, 25.3,  38.9,  66.0,  79.2])    # kg-CO₂
letters          = np.array(list('ABCDEFG'))                            # index 0 → “A”, …

# ─────────────────────────────────────────────────────────────────────────────
# 1.  Helper: numeric Series → array of letters (vectorised, NaN-safe)
# ─────────────────────────────────────────────────────────────────────────────
def to_letter(series: pd.Series, thresholds: np.ndarray) -> pd.Series:
    idx = np.searchsorted(thresholds, series.fillna(np.inf), side='right')
    out = pd.Series(letters[idx], index=series.index, dtype='object')
    out[series.isna()] = np.nan
    return out

df_clean['cert_cons'] = to_letter(df_clean['consumption_in_mkw/m2_year'],
                                  cons_thresholds)
df_clean['cert_emis'] = to_letter(df_clean['emissions_in_kgco2/m2_year'],
                                  emis_thresholds)

# ─────────────────────────────────────────────────────────────────────────────
# 2.  Global class = “worst” (alphabetically later) of the two partial letters
# ─────────────────────────────────────────────────────────────────────────────
rank = {l: i for i, l in enumerate(letters)}     # A→0, …, G→6
reverse_rank = {v: k for k, v in rank.items()}

worst_rank = pd.DataFrame({
    'c': df_clean['cert_cons'].map(rank),
    'e': df_clean['cert_emis'].map(rank)
}).max(axis=1, skipna=True)

df_clean['Certificado energético'] = worst_rank.map(reverse_rank)

# ─────────────────────────────────────────────────────────────────────────────
# 3.  Clean-up: drop the numeric inputs + helper cols
# ─────────────────────────────────────────────────────────────────────────────
df_clean.drop(columns=[
    'consumption_in_mkw/m2_year',
    'emissions_in_kgco2/m2_year',
    'cert_cons', 'cert_emis'
], inplace=True)

# ─────────────────────────────────────────────────────────────────────────────
# 4.  Quick check
# ─────────────────────────────────────────────────────────────────────────────
print(df_clean['Certificado energético'].value_counts(dropna=False))


Certificado energético
E    413
D    301
G    108
C    106
A     64
F     50
B     24
Name: count, dtype: int64


In [122]:
df_clean.rename(columns={'year_built': 'Año de construccion'}, inplace=True)


In [123]:
import numpy as np
import re

# ── 1  filter rows whose address contains any common street keyword ─────────
STREET_REGEX = (
    r'\b('
    r'calle|avenida|avda\.?|paseo|pso\.?|plaza|pl\.?|rambla|carrer|camino|cami'
    r'|ronda|via|vía|traves[ií]a|callejon|callejón|glorieta'
    r')\b'
)

mask_street = df_clean['adress'].str.contains(STREET_REGEX, case=False, na=False, regex=True)
df_clean = df_clean.loc[mask_street].copy()

# ── 2  ensure every remaining address ends with a house number (1–299) ─────
def ensure_number(addr: str) -> str:
    if re.search(r'\d+', addr):
        return addr.strip()
    num = np.random.randint(1, 300)          # adjust range here if you wish
    return f"{addr.strip()} {num}"

df_clean['Ubicacion: calle y numero'] = df_clean['adress'].apply(ensure_number)

# ── 3  drop the old column so the schema matches Dataset 1 ──────────────────
df_clean.drop(columns=['adress'], inplace=True)

# ── 4  quick sanity check ───────────────────────────────────────────────────
print(df_clean[['Ubicacion: calle y numero']].head())
print(f"Rows kept after filtering: {len(df_clean):,}")


          Ubicacion: calle y numero
3              rambla del Raval 173
14        avenida del Paral·lel 149
68              calle sant marti 80
135  calle de Santa Joana d'Arc 213
147            calle d'Ortigosa 203
Rows kept after filtering: 821


/var/folders/5r/5_r4h6857zz40638vhy248s40000gn/T/ipykernel_36490/592485396.py:12: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_street = df_clean['adress'].str.contains(STREET_REGEX, case=False, na=False, regex=True)


In [124]:
df_clean.rename(columns={'city': 'Localidad'}, inplace=True)


In [125]:
df_clean.rename(columns={'neighborhood': 'Barrio'}, inplace=True)


In [126]:
df_clean.rename(columns={'sq_m_built': 'Superficie total (m2)'}, inplace=True)


In [127]:
import re
import numpy as np

# 1️⃣  Pull the fee if it’s written in the ad copy
fee_pattern = re.compile(
    r'(gastos\s+de\s+comunidad|comunidad)\s*[:\-]?\s*(\d{2,4})\s*€',
    flags=re.I
)

def fee_from_text(text):
    if isinstance(text, str):
        m = fee_pattern.search(text)
        if m:
            return float(m.group(2))
    return np.nan

# 2️⃣  Heuristic estimate when the ad doesn’t specify it
def fee_estimate(row):
    base = 0.70                                 # bare-bones fee per m²
    if row.get('Ascensor'):       base += 0.15
    if row.get('Piscina'):        base += 0.25
    if row.get('Jardin'):         base += 0.10
    if row.get('garage'):         base += 0.10
    if row.get('Calefacción') and row.get('central_heating', '') == 'central':
        base += 0.35

    m2 = row.get('Superficie total (m2)', np.nan)
    if np.isnan(m2):
        return np.nan             # can’t guess without floor area

    fee = base * m2
    fee *= np.random.uniform(0.95, 1.05)   # ± 5 % jitter
    return round(fee)

# 3️⃣  Combine: declared fee if found, else estimate
def community_fee(row):
    declared = fee_from_text(row['description'])
    return declared if not np.isnan(declared) else fee_estimate(row)

df_clean['Gastos de la comunidad (€ por mes)'] = df_clean.apply(community_fee, axis=1)

# 4️⃣  Inspect the result
print(df_clean['Gastos de la comunidad (€ por mes)'].describe())
print(df_clean[['Ubicacion: calle y numero', 'Gastos de la comunidad (€ por mes)']].head())


count    821.000000
mean     109.934227
std       80.013859
min       26.000000
25%       59.000000
50%       83.000000
75%      130.000000
max      561.000000
Name: Gastos de la comunidad (€ por mes), dtype: float64
          Ubicacion: calle y numero  Gastos de la comunidad (€ por mes)
3              rambla del Raval 173                                63.0
14        avenida del Paral·lel 149                                62.0
68              calle sant marti 80                                38.0
135  calle de Santa Joana d'Arc 213                                65.0
147            calle d'Ortigosa 203                               137.0


In [128]:
import numpy as np
import pandas as pd

# ── helper that maps one row to the 5 Visitia categories ────────────────────
def classify_heating(row) -> str:
    """
    Uses Calefacción (bool), central_heating (str/bool) and heating_type (str)
    to return:
      • Caldera de gas natural
      • Bomba de calor (Aerotermia)
      • Caldera de biomasa
      • Sin calefaccion
      • Otro tipo de calefaccion
    """
    # 0) explicit “no heating”
    if not row.get('Calefacción', False) or str(row.get('central_heating')).lower() == 'no':
        return 'Sin calefaccion'

    # 1) detailed type if provided
    ht = str(row.get('heating_type', '')).lower()

    if ht in {'natural', 'gas', 'gas natural'}:
        return 'Caldera de gas natural'

    if ht in {'air', 'aerotermia', 'heat pump', 'bomba de calor'}:
        return 'Bomba de calor (Aerotermia)'

    if ht in {'biomass', 'biomasa', 'pellet', 'wood'}:
        return 'Caldera de biomasa'

    # 2) fallback for electric, fuel-oil, unknown, etc.
    return 'Otro tipo de calefaccion'


# 1️⃣  create the new Visitia column
df_clean['Tipo de calefaccion'] = df_clean.apply(classify_heating, axis=1)

# 2️⃣  drop only the auxiliary columns (keep Calefacción)
df_clean.drop(columns=['central_heating', 'heating_type'], inplace=True)

# 3️⃣  quick view of the distribution
print(df_clean['Tipo de calefaccion'].value_counts(dropna=False))


Tipo de calefaccion
Caldera de gas natural         561
Otro tipo de calefaccion       160
Bomba de calor (Aerotermia)     99
Sin calefaccion                  1
Name: count, dtype: int64


In [129]:
# count each unique value (True/False or NaN) in the Calefacción column
print(df_clean['Calefacción'].value_counts(dropna=False))


Calefacción
True     820
False      1
Name: count, dtype: int64


In [130]:
df_clean.rename(columns={'orientation': 'orientacion'}, inplace=True)


In [131]:
print(df_clean.columns)

Index(['id', 'Tipo de vivienda', 'Barrio', 'Localidad', 'price',
       'Superficie total (m2)', 'Superficie util (m2)', 'Habitaciones',
       'Baños', 'Año de construccion', 'Vistas al exterior', 'Ascensor',
       'Terraza', 'Balcon', 'garage', 'Piscina', 'Jardin',
       'Aire acondicionado', 'Calefacción', 'orientacion', 'description',
       'Estado', 'Interes', 'Precio de compra (€)', 'Precio de alquiler',
       'Numero de Planta', 'Tipo de planta', 'Certificado energético',
       'Ubicacion: calle y numero', 'Gastos de la comunidad (€ por mes)',
       'Tipo de calefaccion'],
      dtype='object')


In [132]:
import pandas as pd
import numpy as np

# helper to choose the appropriate price column (sale price preferred over rent)
def choose_price(row):
    if not pd.isna(row['Precio de compra (€)']):
        return row['Precio de compra (€)']
    if not pd.isna(row['Precio de alquiler']):
        return row['Precio de alquiler']
    return np.nan                      # no usable price

# vectorised computation
area = df_clean['Superficie total (m2)']
price = df_clean.apply(choose_price, axis=1)

df_clean['precio por metro cuadrado (€)'] = np.where(
    (area > 0) & ~pd.isna(price),
    (price / area).round(2),           # two-decimal precision
    np.nan
)

# quick look
print(df_clean['precio por metro cuadrado (€)'].describe())


count      821.000000
mean      2941.267795
std       2634.045769
min          3.840000
25%         22.210000
50%       3076.920000
75%       4626.870000
max      15662.650000
Name: precio por metro cuadrado (€), dtype: float64


In [133]:
df_clean.rename(columns={'description': 'Nota'}, inplace=True)


In [134]:
print(df_clean.columns)

Index(['id', 'Tipo de vivienda', 'Barrio', 'Localidad', 'price',
       'Superficie total (m2)', 'Superficie util (m2)', 'Habitaciones',
       'Baños', 'Año de construccion', 'Vistas al exterior', 'Ascensor',
       'Terraza', 'Balcon', 'garage', 'Piscina', 'Jardin',
       'Aire acondicionado', 'Calefacción', 'orientacion', 'Nota', 'Estado',
       'Interes', 'Precio de compra (€)', 'Precio de alquiler',
       'Numero de Planta', 'Tipo de planta', 'Certificado energético',
       'Ubicacion: calle y numero', 'Gastos de la comunidad (€ por mes)',
       'Tipo de calefaccion', 'precio por metro cuadrado (€)'],
      dtype='object')


In [135]:
# saves the file in the folder where your notebook / script is running
df_clean.to_csv("df_clean_processed.csv", index=False)
print("File saved as df_clean_processed.csv")


File saved as df_clean_processed.csv


In [136]:
# 🍿 1.  Unique municipalities (Localidad)
print("\n--- Localidades ---")
print(df_clean['Localidad'].value_counts(dropna=False))

# 🍿 2.  Unique barrios for the largest city in the set
largest_city = df_clean['Localidad'].value_counts().idxmax()
print(f"\n--- Barrios within {largest_city} ---")
print(df_clean.loc[df_clean['Localidad'] == largest_city, 'Barrio']
          .value_counts(dropna=False)
          .head(50))          # adjust the head() count if you need more



--- Localidades ---
Localidad
Barcelona                   718
Badalona                     57
Santa Coloma de Gramenet     23
Hospitalet de Llobregat      21
Sant Adriá de Besós           2
Name: count, dtype: int64

--- Barrios within Barcelona ---
Barrio
Sant Gervasi - Galvany                          74
El Raval                                        71
La Dreta de l'Eixample                          43
Pedralbes                                       40
El Gòtic                                        38
Sant Andreu                                     29
El Poble Sec - Parc de Montjuïc                 25
Les Corts                                       22
Hostafrancs                                     19
Sant Martí de Provençals                        18
Sant Pere - Santa Caterina i la Ribera          18
El Camp de l'Arpa del Clot                      18
El Poblenou                                     16
La Nova Esquerra de l'Eixample                  15
La Sagrada Família          

In [137]:
import numpy as np
import pandas as pd

# 1️⃣  Provincia: all current localidades share the same province
df_clean['Provincia'] = 'Barcelona'          # one-liner, but keeps the column explicit

# 2️⃣  Distrito: map only for rows where Localidad == 'Barcelona'
barrio_to_distrito = {
    # Ciutat Vella
    'El Raval': 'Ciutat Vella',
    'El Gòtic': 'Ciutat Vella',
    'Sant Pere - Santa Caterina i la Ribera': 'Ciutat Vella',
    # Eixample
    "La Dreta de l'Eixample": 'Eixample',
    "L'Antiga Esquerra de l'Eixample": 'Eixample',
    'La Nova Esquerra de l\'Eixample': 'Eixample',
    'La Sagrada Família': 'Eixample',
    'Sant Antoni': 'Eixample',
    # Les Corts
    'Les Corts': 'Les Corts',
    'Pedralbes': 'Les Corts',
    # Sarrià-Sant Gervasi
    'Sarrià': 'Sarrià-Sant Gervasi',
    'Les Tres Torres': 'Sarrià-Sant Gervasi',
    'Sant Gervasi - Galvany': 'Sarrià-Sant Gervasi',
    'Sant Gervasi - La Bonanova': 'Sarrià-Sant Gervasi',
    'El Putxet i el Farró': 'Sarrià-Sant Gervasi',
    # Gràcia
    'Vila de Gràcia': 'Gràcia',
    "El Camp d'En Grassot i Gràcia Nova": 'Gràcia',
    'Vallcarca i els Penitents': 'Gràcia',
    # Horta-Guinardó
    'Horta': 'Horta-Guinardó',
    'El Guinardó': 'Horta-Guinardó',
    'La Teixonera': 'Horta-Guinardó',
    'El Carmel': 'Horta-Guinardó',
    # Nou Barris
    'Les Roquetes': 'Nou Barris',
    'Verdun': 'Nou Barris',
    'La Prosperitat': 'Nou Barris',
    'Vilapicina i la Torre Llobeta': 'Nou Barris',
    'Canyelles': 'Nou Barris',
    # Sant Andreu
    'Sant Andreu': 'Sant Andreu',
    'El Bon Pastor': 'Sant Andreu',
    'Navas': 'Sant Andreu',
    'La Sagrera': 'Sant Andreu',
    'Trinitat Vella': 'Sant Andreu',
    # Sant Martí
    'El Poblenou': 'Sant Martí',
    'Diagonal Mar i el Front Marítim del Poblenou': 'Sant Martí',
    'Provençals del Poblenou': 'Sant Martí',
    'Sant Martí de Provençals': 'Sant Martí',
    "El Camp de l'Arpa del Clot": 'Sant Martí',
    'El Clot': 'Sant Martí',
    'La Verneda i la Pau': 'Sant Martí',
    # Sants-Montjuïc
    'El Poble Sec - Parc de Montjuïc': 'Sants-Montjuïc',
    'Sants': 'Sants-Montjuïc',
    'Sants - Badal': 'Sants-Montjuïc',
    'Hostafrancs': 'Sants-Montjuïc',
    'La Marina del Port': 'Sants-Montjuïc',
}

def assign_distrito(row):
    if row['Localidad'] != 'Barcelona':
        return np.nan                      # districts only apply inside Barcelona
    return barrio_to_distrito.get(row['Barrio'], np.nan)

df_clean['Distrito'] = df_clean.apply(assign_distrito, axis=1)

# 3️⃣  Quick sanity check
print(df_clean[['Localidad', 'Barrio', 'Provincia', 'Distrito']].head())
print('\nDistrito coverage:\n', df_clean['Distrito'].value_counts(dropna=False))


     Localidad                                  Barrio  Provincia  \
3    Barcelona                                El Raval  Barcelona   
14   Barcelona                                El Raval  Barcelona   
68   Barcelona                                El Raval  Barcelona   
135  Barcelona                                   Horta  Barcelona   
147  Barcelona  Sant Pere - Santa Caterina i la Ribera  Barcelona   

           Distrito  
3      Ciutat Vella  
14     Ciutat Vella  
68     Ciutat Vella  
135  Horta-Guinardó  
147    Ciutat Vella  

Distrito coverage:
 Distrito
NaN                    156
Ciutat Vella           127
Sarrià-Sant Gervasi    107
Eixample                96
Sant Martí              86
Sants-Montjuïc          65
Les Corts               62
Sant Andreu             39
Horta-Guinardó          33
Gràcia                  25
Nou Barris              25
Name: count, dtype: int64


In [138]:
import numpy as np
import pandas as pd

# 1️⃣  Postcode lookup ­– Barcelona barrios (complete for current dataset)
barri_zip = {
    # Ciutat Vella
    'El Raval':                                  '08001',
    'El Gòtic':                                  '08002',
    'Sant Pere - Santa Caterina i la Ribera':    '08003',
    'La Barceloneta':                            '08003',

    # Eixample
    "La Dreta de l'Eixample":                    '08007',
    "L'Antiga Esquerra de l'Eixample":           '08008',
    "La Nova Esquerra de l'Eixample":            '08015',
    'La Sagrada Família':                        '08013',
    'Sant Antoni':                               '08001',
    'El Fort Pienc':                             '08013',
    'El Baix Guinardó':                          '08025',

    # Les Corts
    'Les Corts':                                 '08028',
    'Pedralbes':                                 '08034',
    'La Maternitat i Sant Ramon':                '08028',

    # Sarrià-Sant Gervasi
    'Sarrià':                                    '08017',
    'Les Tres Torres':                           '08017',
    'Sant Gervasi - Galvany':                    '08021',
    'Sant Gervasi - La Bonanova':                '08022',
    'El Putxet i el Farró':                      '08023',
    'Vallvidrera - El Tibidabo i les Planes':    '08017',

    # Gràcia
    'Vila de Gràcia':                            '08012',
    "El Camp d'En Grassot i Gràcia Nova":        '08025',
    'Vallcarca i els Penitents':                 '08023',
    'La Salut':                                  '08024',
    'El Coll':                                   '08023',

    # Horta-Guinardó
    'Horta':                                     '08031',
    'El Guinardó':                               '08041',
    'La Teixonera':                              '08035',
    'El Carmel':                                 '08032',
    'Can Baró':                                  '08041',
    "La Font d'En Fargues":                      '08032',
    'Sant Genís Dels Agudells - Montbau':        '08035',

    # Nou Barris
    'Les Roquetes':                              '08042',
    'Verdun':                                    '08042',
    'La Prosperitat':                            '08033',
    'Vilapicina i la Torre Llobeta':             '08031',
    'Canyelles':                                 '08042',
    'Can Peguera - El Turó de la Peira':         '08031',
    'La Guineueta':                              '08042',
    'La Trinitat Nova':                          '08033',

    # Sant Andreu
    'Sant Andreu':                               '08030',
    'El Bon Pastor':                             '08030',
    'Navas':                                     '08027',
    'La Sagrera':                                '08027',
    'Trinitat Vella':                            '08030',
    'La Trinitat Vella':                         '08030',
    'El Congrés i els Indians':                  '08027',

    # Sant Martí
    'El Poblenou':                               '08005',
    'La Vila Olímpica del Poblenou':             '08005',
    'Diagonal Mar i el Front Marítim del Poblenou': '08019',
    'Provençals del Poblenou':                   '08019',
    'Sant Martí de Provençals':                  '08020',
    "El Camp de l'Arpa del Clot":                '08026',
    'El Clot':                                   '08018',
    'La Verneda i la Pau':                       '08020',
    'El Besòs':                                  '08019',
    'El Parc i la Llacuna del Poblenou':         '08005',

    # Sants-Montjuïc
    'El Poble Sec - Parc de Montjuïc':           '08004',
    'Sants':                                     '08028',
    'Sants - Badal':                             '08028',
    'Hostafrancs':                               '08014',
    'La Marina del Port':                        '08038',
    'La Marina del Prat Vermell':                '08038',
    'La Bordeta':                                '08014',
    'La Font de la Guatlla':                     '08038',
}

# 2️⃣  Surrounding municipalities – one postcode each
localidad_zip = {
    'Barcelona':                   np.nan,   # filled via barrio
    'Badalona':                    '08912',
    'Santa Coloma de Gramenet':    '08921',
    'Hospitalet de Llobregat':     '08901',
    'Sant Adriá de Besós':         '08930',
}

# 3️⃣  Assign Código postal
def assign_cp(row):
    if row['Localidad'] == 'Barcelona':
        return barri_zip.get(row['Barrio'], np.nan)
    return localidad_zip.get(row['Localidad'], np.nan)

df_clean['Codigo postal'] = df_clean.apply(assign_cp, axis=1)

# 4️⃣  Verify
print(df_clean[['Localidad', 'Barrio', 'Codigo postal']].head())
print('\nMissing postcodes:', df_clean['Codigo postal'].isna().sum())


     Localidad                                  Barrio Codigo postal
3    Barcelona                                El Raval         08001
14   Barcelona                                El Raval         08001
68   Barcelona                                El Raval         08001
135  Barcelona                                   Horta         08031
147  Barcelona  Sant Pere - Santa Caterina i la Ribera         08003

Missing postcodes: 0


In [139]:
import numpy as np
import pandas as pd

# 1️⃣  Which barrios (and towns) are on the coastline?
coastal_barrios = {
    'La Barceloneta',
    'El Poblenou',
    'La Vila Olímpica del Poblenou',
    'Diagonal Mar i el Front Marítim del Poblenou',
    'El Besòs',
    'El Parc i la Llacuna del Poblenou',         # touches the seafront park
    'La Marina del Port',                        # Port zone
    'La Marina del Prat Vermell',
}

coastal_towns = {'Badalona', 'Sant Adriá de Besós'}   # whole municipality is coastal

# 2️⃣  Function to decide Yes/No per row
rng = np.random.default_rng()        # independent generator

def sea_view(row):
    loc  = row['Localidad']
    barr = row['Barrio']
    coastal = (loc in coastal_towns) or (barr in coastal_barrios)
    if not coastal:
        return 'No'
    # 50-50 chance for coastal listings
    return rng.choice(['Sí', 'No'])

# 3️⃣  Create the new column
df_clean['Vistas al mar'] = df_clean.apply(sea_view, axis=1)

# 4️⃣  Quick check
print(df_clean['Vistas al mar'].value_counts(dropna=False))


Vistas al mar
No    768
Sí     53
Name: count, dtype: int64


In [140]:
import numpy as np

rng = np.random.default_rng()   # independent random generator

# 1️⃣  Armarios empotrados
def choose_armarios(row):
    prob_yes = 0.8 if (row['Estado'] in ['a estrenar', 'reformado'] or
                       row['Interes'] == 'obra nueva') else 0.4
    return rng.choice(['Sí', 'No'], p=[prob_yes, 1 - prob_yes])

df_clean['Armarios empotrados'] = df_clean.apply(choose_armarios, axis=1)

# 2️⃣  Vivienda accesible
def choose_accesible(row):
    prob_yes = 0.7 if row.get('Ascensor') else 0.2
    return rng.choice(['Sí', 'No'], p=[prob_yes, 1 - prob_yes])

df_clean['Vivienda accesible'] = df_clean.apply(choose_accesible, axis=1)

# 3️⃣  Trastero o bodega
def choose_trastero(row):
    prob_yes = 0.6 if row.get('garage') else 0.3
    return rng.choice(['Sí', 'No'], p=[prob_yes, 1 - prob_yes])

df_clean['Trastero o bodega'] = df_clean.apply(choose_trastero, axis=1)




In [141]:
import numpy as np
import pandas as pd

rng = np.random.default_rng()   # independent generator

# 1️⃣  Muebles ─ furnished homes are much more common on the rental market
def choose_muebles(row):
    prob_yes = 0.75 if row['Interes'] == 'alquilar' else 0.30
    return rng.choice(['Sí', 'No'], p=[prob_yes, 1 - prob_yes])

df_clean['Muebles'] = df_clean.apply(choose_muebles, axis=1)

# 2️⃣  Vigilancia ─ more likely when the building offers premium services
def choose_vigilancia(row):
    premium_flags = sum([
        row.get('Ascensor', False),
        row.get('Piscina',  False),
        row.get('garage',   False),
        row.get('Jardin',   False)
    ])
    # baseline 15 % → climbs to 70 % when several amenities are present
    prob_yes = 0.15 + 0.15 * premium_flags          # 0.15 → 0.75
    prob_yes = min(prob_yes, 0.75)
    return rng.choice(['Sí', 'No'], p=[prob_yes, 1 - prob_yes])

df_clean['Vigilancia'] = df_clean.apply(choose_vigilancia, axis=1)

# 3️⃣  Vivienda de lujo ─ flagged when price or €/m² is high
high_price      = df_clean['price'] > 600_000
high_ppm2       = df_clean['precio por metro cuadrado (€)'] > 4500

def choose_lujo(row):
    if high_price.loc[row.name] or high_ppm2.loc[row.name]:
        prob_yes = 0.80          # strong chance of “luxury”
    else:
        prob_yes = 0.10          # only occasional upscale exceptions
    return rng.choice(['Sí', 'No'], p=[prob_yes, 1 - prob_yes])

df_clean['Vivienda de lujo'] = df_clean.apply(choose_lujo, axis=1)

# Quick sanity check
print(df_clean[['Muebles', 'Vigilancia', 'Vivienda de lujo']].value_counts())


Muebles  Vigilancia  Vivienda de lujo
No       No          No                  166
Sí       No          No                  154
No       No          Sí                  131
Sí       No          Sí                   95
No       Sí          No                   86
Sí       Sí          No                   78
No       Sí          Sí                   61
Sí       Sí          Sí                   50
Name: count, dtype: int64


In [142]:
# Save the current DataFrame to a CSV in your working folder (shown by os.getcwd())
df_clean.to_csv("df.csv", index=False)



In [143]:
import numpy as np
import pandas as pd

rng = np.random.default_rng()

def build_unit_string(row):
    piso = row['Numero de Planta']          # already an int (can be −1 .. 25)

    # 1️⃣  Normalise ground/basement wording
    if pd.isna(piso):
        piso_str = 's/n'                    # unknown floor
    elif piso <= 0:
        piso_str = 'Bajo' if piso == 0 else f'Sótano {abs(int(piso))}'
    else:
        piso_str = f'{int(piso)}º'

    # 2️⃣  Random door letter
    door_letter = rng.choice(list('ABCD'))

    # 3️⃣  Optional portal number for larger blocks
    need_portal = (row.get('Ascensor', False) or (piso and piso >= 4))
    portal = f' Portal {rng.integers(1, 4)}' if need_portal else ''

    return f'{piso_str} {door_letter}{portal}'

df_clean['Ubicacion: Piso y número, letra o portal'] = df_clean.apply(build_unit_string, axis=1)

# Quick peek
print(df_clean[['Numero de Planta', 'Ubicacion: Piso y número, letra o portal']].head())


     Numero de Planta Ubicacion: Piso y número, letra o portal
3                   3                                     3º A
14                  5                            5º A Portal 2
68                  1                            1º C Portal 1
135                 1                            1º B Portal 3
147                 1                            1º B Portal 1


In [144]:
# mapping municipio → (ID, Nombre)
agency_lookup = {
    'Barcelona':                   (1, 'Lucas Fox Barcelona'),
    'Badalona':                    (2, 'VIP House Badalona'),
    'Santa Coloma de Gramenet':    (3, 'Fincas Mirafer'),
    'Hospitalet de Llobregat':     (4, 'Punt Buenos Aires BCN'),
    'Sant Adriá de Besós':         (5, 'Asa Habitatges'),
}

df_clean[['ID de inmobiliaria', 'Nombre de inmobiliaria']] = (
    df_clean['Localidad']
      .apply(lambda loc: agency_lookup[loc])
      .apply(pd.Series)
)



In [145]:
import re

def build_nombre(row):
    tipo  = str(row['Tipo de vivienda']).capitalize()        # e.g. “Piso”
    calle = str(row['Ubicacion: calle y numero']).title()    # tidy street string
    # Optional: shorten “Calle” → “C.” etc.
    calle = re.sub(r'\bCalle\b', 'C.', calle, flags=re.I)
    calle = re.sub(r'\bAvenida\b', 'Av.', calle, flags=re.I)
    return f"{tipo} {calle}"

df_clean['Nombre vivienda'] = df_clean.apply(build_nombre, axis=1)

# quick peek
print(df_clean[['Nombre vivienda']].head())


                                  Nombre vivienda
3           Flat / apartment Rambla Del Raval 173
14         Flat / apartment Av. Del Paral·Lel 149
68              Flat / apartment C. Sant Marti 80
135  Flat / apartment C. De Santa Joana D'Arc 213
147            Flat / apartment C. D'Ortigosa 203


In [146]:
import pandas as pd

# 1️⃣  Create a repeatable barrio → ID mapping
barrio_ids = {b: i for i, b in enumerate(df_clean['Barrio'].dropna().unique(), start=1)}

# 2️⃣  Add the two new columns
df_clean['ID mapa ubicacion']      = df_clean['Barrio'].map(barrio_ids)
df_clean['Nombre mapa de Ubicacion'] = df_clean['Barrio']        # copy as-is

# 3️⃣  Quick peek
print(df_clean[['Barrio', 'ID mapa ubicacion', 'Nombre mapa de Ubicacion']].head())


                                     Barrio  ID mapa ubicacion  \
3                                  El Raval                  1   
14                                 El Raval                  1   
68                                 El Raval                  1   
135                                   Horta                  2   
147  Sant Pere - Santa Caterina i la Ribera                  3   

                   Nombre mapa de Ubicacion  
3                                  El Raval  
14                                 El Raval  
68                                 El Raval  
135                                   Horta  
147  Sant Pere - Santa Caterina i la Ribera  


In [147]:
df_clean.to_csv("final_df.csv", index=False)

In [148]:
# number of columns
n_cols = df_clean.shape[1]     # or len(df_clean.columns)

# null counts per column
nulls = df_clean.isna().sum()  # .isnull() works the same

print(f"Columns in df_clean: {n_cols}")
print(nulls)


Columns in df_clean: 48
id                                            0
Tipo de vivienda                              0
Barrio                                        0
Localidad                                     0
price                                         0
Superficie total (m2)                         0
Superficie util (m2)                          0
Habitaciones                                  0
Baños                                         0
Año de construccion                           0
Vistas al exterior                            0
Ascensor                                      0
Terraza                                       0
Balcon                                        0
garage                                        0
Piscina                                       0
Jardin                                        0
Aire acondicionado                            0
Calefacción                                   0
orientacion                                   0
Nota            

In [149]:
# drop the column named "price" and update the DataFrame
df_clean = df_clean.drop(columns=["price"])      # returns a new DataFrame

# —or— do it in-place:
# df_clean.drop(columns=["price"], inplace=True)


In [150]:
# number of rows in df_clean
n_rows = df_clean.shape[0]      # or simply len(df_clean)

print(f"Rows in df_clean: {n_rows}")


Rows in df_clean: 821


In [151]:
df_clean.to_csv("more_rentals_df.csv", index=False)

In [152]:
# List of columns with "Sí"/"No" values
yes_no_columns = [
    'Vistas al mar',
    'Armarios empotrados',
    'Vivienda accesible',
    'Trastero o bodega',
    'Muebles',
    'Vigilancia',
    'Vivienda de lujo'
]

# Convert to boolean: True for "Sí", False for "No"
df_clean[yes_no_columns] = df_clean[yes_no_columns].applymap(lambda x: x == "Sí")


/var/folders/5r/5_r4h6857zz40638vhy248s40000gn/T/ipykernel_36490/1267715369.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_clean[yes_no_columns] = df_clean[yes_no_columns].applymap(lambda x: x == "Sí")
